In [1]:
import pandas as pd
from poste import geocode_nominatim as gn
from poste import ItalianAddressValidator as iav
import importlib
import ipyfilechooser
from openpyxl import load_workbook
import re

In [2]:
importlib.reload(gn)

<module 'poste.geocode_nominatim' from 'C:\\Users\\User\\PycharmProjects\\geoai\\poste\\geocode_nominatim.py'>

In [3]:
# Global variable to store addresses
addresses = []

# File chooser
chooser = ipyfilechooser.FileChooser()
display(chooser)

FileChooser(path='C:\Users\User\PycharmProjects\geoai', filename='', title='', show_hidden=False, select_desc=…

In [30]:
wb = load_workbook(chooser.value)
sheet = wb.active  # Get active sheet
data = sheet.values  # Read values

# No Pandas

In [31]:
addresses_list = []

for index,row in enumerate(data):
    # print(index,row[0],row[4],row[5])
    if index >0:        
        result = ",".join([row[0],row[12], row[13], row[17], row[18], row[22],row[23]])
        addresses_list.append(result)
        # print(row[1],row[4],row[5])  # Print each row

In [29]:
addresses_list

['VIA COMUNALE DELLE PUGLIE,171,80013,CASALNUOVO DI NAPOLI,NA,NAPOLI',
 'VIA GRAZIA,37/37,98057,MILAZZO,ME,MESSINA',
 'VIA VALLESELLA,24,35046,BORGO VENETO,PD,PADOVA',
 'VIA IRLANDA,11,50063,FIGLINE E INCISA VALDARNO,FI,FIRENZE',
 'VIA MACANNO,SNC,47923,RIMINI,RN,RIMINI',
 'VIA URUGUAY,SNC,58100,GROSSETO,GR,GROSSETO',
 'VIA CEGLIE FRANCAVILLA,44,72013,CEGLIE MESSAPICA,BR,BRINDISI',
 'VIA PIETRO COSSA,2934/B,10151,TORINO,TO,TORINO',
 'VIA EVANGELISTA,36,81025,MARCIANISE,CE,CASERTA',
 'PIAZZA VITTORIA,16,88900,CROTONE,KR,CROTONE',
 'VIA MARE,90,73059,UGENTO,LE,LECCE',
 'VIA GIUSEPPE MAZZINI,2,46034,BORGO VIRGILIO,MN,MANTOVA',
 'STRADA PROVINCIALE 227,227/2,20009,VITTUONE,MI,MILANO',
 'FRAZIONE TRAVERSA,SNC,07052,SAN TEODORO,SS,SASSARI',
 'VIA DEL GIGNORO,19,50135,FIRENZE,FI,FIRENZE',
 'STRADA MORANE,21,41126,MODENA,MO,MODENA',
 'VIA CARLO ALBERTO DALLA CHIESA,7,87017,ROGGIANO GRAVINA,CS,COSENZA',
 'VIALE USA,39/U,71042,CERIGNOLA,FG,FOGGIA',
 'PIAZZA PANICO CARDINALE,4,73039,TRICASE,LE,LE

In [8]:
# Splitting each address into parts
split_addresses = [addr.split() for addr in addresses_list]

In [9]:
# split_addresses

In [10]:
# Finding a pattern to extract columns
def split_address(address):
    parts = address.split()
    
    # Find where the postal code starts (always 5 digits)
    for i, part in enumerate(parts):
        if part.isdigit() and len(part) == 5:
            street = " ".join(parts[:i-1])  # Everything before house number
            number = parts[i-1]             # House number
            postal_code = parts[i]          # Postal code
            city = " ".join(parts[i+1:])    # City name
            return [street, number, postal_code, city]
    
    return [address, "", "", ""]  # Fallback if no match

In [11]:
# Process all addresses
df = pd.DataFrame([split_address(addr) for addr in addresses], columns=['Street', 'Number', 'Postal Code', 'City'])

df

,Street,Number,Postal Code,City


# Spotting Inconsistent Structure

In [12]:
# for address in addresses_list:
#     # Find numbers, allowing for trailing letters (e.g., 12A, 37B)
#     numbers = re.findall(r'\d+[A-Za-z]?', address)
    
#     if len(numbers) < 2:
#         print(f"Skipping (less than 2 numbers): {address}")
#         continue
    
#     first_num = numbers[0]
#     second_num = numbers[1]
#     more_than_two = len(numbers) > 2

#     # Split at the first number
#     parts = re.split(r'\b' + re.escape(first_num) + r'\b', address, 1)
#     first_split = parts[0].strip()
#     rest = parts[1].strip()

#     # Split at the second number
#     parts2 = re.split(r'\b' + re.escape(second_num) + r'\b', rest, 1)
#     second_split = parts2[0].strip()
#     remaining = parts2[1].strip() if len(parts2) > 1 else ''

#     print(f"Address: {address}")
#     print(f"First Part: {first_split}")
#     print(f"Second Part: {second_split}")
#     print(f"Remaining: {remaining}")
#     print(f"More than two numbers: {more_than_two}")
#     print("-" * 50)

In [ ]:
# Extract ID and address separately
def extract_id_address(entry):
    match = re.match(r"(\d+)\s*-\s*(.+)", entry)
    # print(match.group(1))
    if match:
        return match.group(1), match.group(2)
    return None, entry  # If no match, return None for ID

# Convert data into a DataFrame
df = pd.DataFrame([extract_id_address(entry) for entry in data], columns=["ID", "Address"])

## Geocoding

In [13]:
gdf = gn.geocode_addresses(addresses_list)

VIA COMUNALE DELLE PUGLIE 171 80013 CASALNUOVO DI NAPOLI NA NAPOLI: 40.90295424612283, 14.355074120846782
VIA GRAZIA 37/37 98057 MILAZZO ME MESSINA: 38.208113342960544, 15.24493674421997
VIA VALLESELLA 24 35046 BORGO VENETO PD PADOVA: 45.22761885, 11.52203075
VIA IRLANDA 11 50063 FIGLINE E INCISA VALDARNO FI FIRENZE: None, None
VIA MACANNO SNC 47923 RIMINI RN RIMINI: None, None
VIA URUGUAY SNC 58100 GROSSETO GR GROSSETO: None, None
VIA CEGLIE FRANCAVILLA 44 72013 CEGLIE MESSAPICA BR BRINDISI: None, None
VIA PIETRO COSSA 2934/B 10151 TORINO TO TORINO: 45.0970893, 7.63308815
VIA EVANGELISTA 36 81025 MARCIANISE CE CASERTA: 41.03082424958436, 14.308360643769413
PIAZZA VITTORIA 16 88900 CROTONE KR CROTONE: 39.08072235, 17.127087665639397
VIA MARE 90 73059 UGENTO LE LECCE: 39.92139915293526, 18.156463696439605
VIA GIUSEPPE MAZZINI 2 46034 BORGO VIRGILIO MN MANTOVA: 45.11534079138789, 10.785330407494335
STRADA PROVINCIALE 227 227/2 20009 VITTUONE MI MILANO: None, None
FRAZIONE TRAVERSA SNC 07

# GEOCODED VALUES

In [14]:
gdf

,Address,Latitude,Longitude,geometry
0,VIA COMUNALE DELLE PUGLIE 171 80013 CASALNUOVO...,40.902954,14.355074,POINT (14.35507 40.90295)
1,VIA GRAZIA 37/37 98057 MILAZZO ME MESSINA,38.208113,15.244937,POINT (15.24494 38.20811)
2,VIA VALLESELLA 24 35046 BORGO VENETO PD PADOVA,45.227619,11.522031,POINT (11.52203 45.22762)
7,VIA PIETRO COSSA 2934/B 10151 TORINO TO TORINO,45.097089,7.633088,POINT (7.63309 45.09709)
8,VIA EVANGELISTA 36 81025 MARCIANISE CE CASERTA,41.030824,14.308361,POINT (14.30836 41.03082)
9,PIAZZA VITTORIA 16 88900 CROTONE KR CROTONE,39.080722,17.127088,POINT (17.12709 39.08072)
10,VIA MARE 90 73059 UGENTO LE LECCE,39.921399,18.156464,POINT (18.15646 39.9214)
11,VIA GIUSEPPE MAZZINI 2 46034 BORGO VIRGILIO MN...,45.115341,10.785330,POINT (10.78533 45.11534)
14,VIA DEL GIGNORO 19 50135 FIRENZE FI FIRENZE,43.769711,11.298323,POINT (11.29832 43.76971)
15,STRADA MORANE 21 41126 MODENA MO MODENA,44.638919,10.925972,POINT (10.92597 44.63892)


In [15]:
# gdf.to_file('geocoded_nc.shp')

In [16]:
gdf.explore()

# Pandas

In [17]:
# Load the Excel file
df = pd.read_excel(chooser.value, engine="openpyxl")  # Use engine="openpyxl" for .xlsx files

# Display the first few rows
df.head()

,CodiceIdentificativo,Via_Input,Civico_Input,Esponente_Input,CAP_Input,Comune_Input,Frazione_Input,Provincia_Input,Risultato,Modificato,...,SezioneCensimento,Codice_Belfiore,X_METRI,Y_METRI,X_GRADI,Y_GRADI,FonteCoord,QualCoord,Entita,QualitaPTL
0,B06HKB1SUD,VIA COMUNALE DELLE PUGLIE 171,NaN,NaN,80013,CASALNUOVO DI NAPOLI,NaN,NaN,1,0,...,NaN,B905,"00950824,52000","04542296,36000","14,35113000","40,91586000",NaN,NaN,Comune,NaN
1,B0DE7B2QKW,VIA GRAZIA 37 37 37,NaN,NaN,98057,MILAZZO,NaN,ME,1,0,...,NaN,F206,"01046727,49122","04245962,80893","15,24024000","38,21813000",NaN,NaN,Comune,NaN
2,B0GLSB2U60,VIA VALLESELLA 24,NaN,NaN,35046,BORGO VENETO,NaN,PD,1,0,...,NaN,M402,"00698172,0840368725","05011121,0576050090","11,52432060","45,22567960",D,180.0,Civico,NaN
3,3381J58947,VIA IRLANDA 11,NaN,NaN,50064,FIGLINE E INCISA VALDARNO,NaN,FI,1,1,...,NaN,M321,"00699407,21681","04832744,99077","11,46354000","43,62333000",NaN,NaN,Comune,NaN
4,B08O8B2CJ3,VIA MACANNO SNC,NaN,NaN,47923,RIMINI,NaN,RN,0,0,...,NaN,NaN,"00781780,61709","04880497,54729","12,51590000","44,02350000",NaN,NaN,CAP,NaN


In [18]:
df_verifica = df[['CodiceIdentificativo','Via_Input','Civico_Input','CAP_Input','Comune_Input','Provincia_Input']]
df_verifica

,CodiceIdentificativo,Via_Input,Civico_Input,CAP_Input,Comune_Input,Provincia_Input
0,B06HKB1SUD,VIA COMUNALE DELLE PUGLIE 171,NaN,80013,CASALNUOVO DI NAPOLI,NaN
1,B0DE7B2QKW,VIA GRAZIA 37 37 37,NaN,98057,MILAZZO,ME
2,B0GLSB2U60,VIA VALLESELLA 24,NaN,35046,BORGO VENETO,PD
3,3381J58947,VIA IRLANDA 11,NaN,50064,FIGLINE E INCISA VALDARNO,FI
4,B08O8B2CJ3,VIA MACANNO SNC,NaN,47923,RIMINI,RN
...,...,...,...,...,...,...
81,F1MSZF3VWS,TRAV V DI VIA APPIA LATO NAPOLI 5,NaN,4023,FORMIA,LT
82,F0MV9F2OGA,VIA NAZIONALE 12T,NaN,98138,MESSINA,ME
83,F1GJ9F3KZG,VIA ELATE PENTESILEA 16,NaN,24014,PIAZZA BREMBANA,BG
84,B08ROB2CYZ,VIA AMENDOLA GIORGIO 1,NaN,92019,SCIACCA,AG


In [19]:
from pydantic import BaseModel, Field, constr, conint
from typing import List

class Address(BaseModel):
    id_code: constr(min_length=10, max_length=10)  # Fixed length fiscal code
    street: constr(min_length=5)  # Street cannot be empty
    # postal_code: constr(min_length=5)  # Exactly 5 digits as string (preserves leading zeros)
    postal_code: conint(ge=10000, le=99999)  # Assuming valid Italian postal codes
    city: constr(min_length=4)  # City cannot be empty
    province: constr(min_length=2, max_length=2)  # Province code is always 2 characters

class AddressList(BaseModel):
    addresses: List[Address]

In [20]:
# Convert your list to proper format
address_data = {"addresses": [
    {
        "id_code": code,
        "street": street,
        "postal_code": postal,
        "city": city,
        "province": province
    } for code, street, postal, city, province in addresses
]}

# Validate the data
validated_addresses = AddressList(**address_data)

In [21]:
address_data

{'addresses': []}

In [22]:
validated_addresses

AddressList(addresses=[])

In [23]:
## PANDERA
# class Schema(pa.DataFrameModel):
#     id: Series[int]
#     name: Series[str]
#     age: Series[int] = pa.Field(ge=18, le=99)  # Age must be between 18-99

# 'PZA' crea problemi rimpiazzo con 'PIAZZA' e codifico nuovamente

In [24]:
# Select rows where 'name' contains 'PZA'
filtered_df = df[df["Via_Input"].str.contains("PZA", na=False)]
filtered_df

,CodiceIdentificativo,Via_Input,Civico_Input,Esponente_Input,CAP_Input,Comune_Input,Frazione_Input,Provincia_Input,Risultato,Modificato,...,SezioneCensimento,Codice_Belfiore,X_METRI,Y_METRI,X_GRADI,Y_GRADI,FonteCoord,QualCoord,Entita,QualitaPTL
9,F0LC1F2J25,PZA VITTORIA 16,NaN,NaN,88900,CROTONE,NaN,KR,1,0,...,NaN,D122,"01201117,83928","04359362,60321","17,12291000","39,08124000",NaN,NaN,Comune,NaN
18,B0BA4B2OG1,PZA CARDINALE PANICO 4,NaN,NaN,73039,TRICASE,NaN,LE,1,1,...,NaN,L419,"01299865,29752","04463629,54159","18,36048000","39,93104000",NaN,NaN,Comune,NaN
21,F19O6F37FE,PZA DEGLI ALPINI 7A,NaN,NaN,31030,BORSO DEL GRAPPA,NaN,TV,1,0,...,NaN,B061,"00716286,53516","05074924,77378","11,79852000","45,82105000",NaN,NaN,Comune,NaN
23,F1B41F39U1,PZA DEL FONTANILE 14,NaN,NaN,31,ARTENA,NaN,RM,1,0,...,NaN,A449,"00823167,47720","04627155,03790","12,91565000","41,73925000",NaN,NaN,Comune,NaN
39,F1LCPF3TNS,PZA BENIAMINO CHIATTO 2,NaN,NaN,87100,COSENZA,NaN,CS,1,0,...,NaN,D086,"01127468,03114","04372447,60155","16,25702000","39,29313000",NaN,NaN,Comune,NaN
50,F1KJ7F3SBW,PZA FISAC 2,NaN,NaN,22100,COMO,NaN,CO,1,0,...,NaN,C933,"00506513,56782","05071757,92054","09,08710000","45,81076000",NaN,NaN,Comune,NaN
77,J0YC6J3JA2,PZA GIACOMO LEOPARDI 12,NaN,NaN,60012,TRECASTELLI,NaN,AN,3,0,...,NaN,M318,"00831053,57008","04843529,96240","13,10620000","43,67146000",NaN,NaN,Comune,NaN


In [25]:
# Replace 'PZA' with 'PIAZZA' in the 'name' column
filtered_df["Via_Input"] = filtered_df["Via_Input"].str.replace("PZA", "PIAZZA")
filtered_df

C:\Users\User\AppData\Local\Temp\ipykernel_20320\95707981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["Via_Input"] = filtered_df["Via_Input"].str.replace("PZA", "PIAZZA")


,CodiceIdentificativo,Via_Input,Civico_Input,Esponente_Input,CAP_Input,Comune_Input,Frazione_Input,Provincia_Input,Risultato,Modificato,...,SezioneCensimento,Codice_Belfiore,X_METRI,Y_METRI,X_GRADI,Y_GRADI,FonteCoord,QualCoord,Entita,QualitaPTL
9,F0LC1F2J25,PIAZZA VITTORIA 16,NaN,NaN,88900,CROTONE,NaN,KR,1,0,...,NaN,D122,"01201117,83928","04359362,60321","17,12291000","39,08124000",NaN,NaN,Comune,NaN
18,B0BA4B2OG1,PIAZZA CARDINALE PANICO 4,NaN,NaN,73039,TRICASE,NaN,LE,1,1,...,NaN,L419,"01299865,29752","04463629,54159","18,36048000","39,93104000",NaN,NaN,Comune,NaN
21,F19O6F37FE,PIAZZA DEGLI ALPINI 7A,NaN,NaN,31030,BORSO DEL GRAPPA,NaN,TV,1,0,...,NaN,B061,"00716286,53516","05074924,77378","11,79852000","45,82105000",NaN,NaN,Comune,NaN
23,F1B41F39U1,PIAZZA DEL FONTANILE 14,NaN,NaN,31,ARTENA,NaN,RM,1,0,...,NaN,A449,"00823167,47720","04627155,03790","12,91565000","41,73925000",NaN,NaN,Comune,NaN
39,F1LCPF3TNS,PIAZZA BENIAMINO CHIATTO 2,NaN,NaN,87100,COSENZA,NaN,CS,1,0,...,NaN,D086,"01127468,03114","04372447,60155","16,25702000","39,29313000",NaN,NaN,Comune,NaN
50,F1KJ7F3SBW,PIAZZA FISAC 2,NaN,NaN,22100,COMO,NaN,CO,1,0,...,NaN,C933,"00506513,56782","05071757,92054","09,08710000","45,81076000",NaN,NaN,Comune,NaN
77,J0YC6J3JA2,PIAZZA GIACOMO LEOPARDI 12,NaN,NaN,60012,TRECASTELLI,NaN,AN,3,0,...,NaN,M318,"00831053,57008","04843529,96240","13,10620000","43,67146000",NaN,NaN,Comune,NaN


In [26]:
# Extract multiple columns as a list of lists
multiple_columns = filtered_df[["Via_Input","CAP_Input","Comune_Input","Provincia_Input"]].values.tolist()
multiple_columns

[['PIAZZA VITTORIA 16', 88900, 'CROTONE', 'KR'],
 ['PIAZZA CARDINALE PANICO 4', 73039, 'TRICASE', 'LE'],
 ['PIAZZA DEGLI ALPINI 7A', 31030, 'BORSO DEL GRAPPA', 'TV'],
 ['PIAZZA DEL FONTANILE 14', 31, 'ARTENA', 'RM'],
 ['PIAZZA BENIAMINO CHIATTO 2', 87100, 'COSENZA', 'CS'],
 ['PIAZZA FISAC 2', 22100, 'COMO', 'CO'],
 ['PIAZZA GIACOMO LEOPARDI 12', 60012, 'TRECASTELLI', 'AN']]

In [27]:
# Original format but with each sublist joined into a single string
formatted_rows = [', '.join(map(str, sublist)) for sublist in multiple_columns]

In [28]:
formatted_rows

['PIAZZA VITTORIA 16, 88900, CROTONE, KR',
 'PIAZZA CARDINALE PANICO 4, 73039, TRICASE, LE',
 'PIAZZA DEGLI ALPINI 7A, 31030, BORSO DEL GRAPPA, TV',
 'PIAZZA DEL FONTANILE 14, 31, ARTENA, RM',
 'PIAZZA BENIAMINO CHIATTO 2, 87100, COSENZA, CS',
 'PIAZZA FISAC 2, 22100, COMO, CO',
 'PIAZZA GIACOMO LEOPARDI 12, 60012, TRECASTELLI, AN']

# GEOCODING ancora

In [29]:
gdf_pza = gn.geocode_addresses(formatted_rows)

PIAZZA VITTORIA 16, 88900, CROTONE, KR: 39.08072235, 17.127087665639397
PIAZZA CARDINALE PANICO 4, 73039, TRICASE, LE: None, None
PIAZZA DEGLI ALPINI 7A, 31030, BORSO DEL GRAPPA, TV: 45.820917699999995, 11.800631334553259
PIAZZA DEL FONTANILE 14, 31, ARTENA, RM: None, None
PIAZZA BENIAMINO CHIATTO 2, 87100, COSENZA, CS: None, None
PIAZZA FISAC 2, 22100, COMO, CO: 45.7901235, 9.0825724096939
PIAZZA GIACOMO LEOPARDI 12, 60012, TRECASTELLI, AN: None, None


In [30]:
gdf_pza.explore()

In [31]:
gdf_pza

,Address,Latitude,Longitude,geometry
0,"PIAZZA VITTORIA 16, 88900, CROTONE, KR",39.080722,17.127088,POINT (17.12709 39.08072)
2,"PIAZZA DEGLI ALPINI 7A, 31030, BORSO DEL GRAPP...",45.820918,11.800631,POINT (11.80063 45.82092)
5,"PIAZZA FISAC 2, 22100, COMO, CO",45.790123,9.082572,POINT (9.08257 45.79012)


In [22]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
import re

# Initialize Nominatim geocoder
geolocator = Nominatim(user_agent="geo_app123")

# Sample data: list of strings with IDs
data = [
    "12345 - 1600 Amphitheatre Parkway, Mountain View, CA",
    "67890 - 1 Apple Park Way, Cupertino, CA",
    "54321 - 350 5th Ave, New York, NY"
]

# Extract ID and address separately
def extract_id_address(entry):
    match = re.match(r"(\d+)\s*-\s*(.+)", entry)
    # print(match.group(1))
    if match:
        return match.group(1), match.group(2)
    return None, entry  # If no match, return None for ID

# Convert data into a DataFrame
df = pd.DataFrame([extract_id_address(entry) for entry in data], columns=["ID", "Address"])

# Function to geocode an address with retry logic
def geocode_address(address):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
    except GeocoderTimedOut:
        time.sleep(1)
        return geocode_address(address)  # Retry on timeout
    return None, None  # Return None if geocoding fails

# Apply geocoding to the addresses
df["Latitude"], df["Longitude"] = zip(*df["Address"].apply(geocode_address))

# Print results
df

12345
67890
54321


,ID,Address,Latitude,Longitude
0,12345,"1600 Amphitheatre Parkway, Mountain View, CA",37.422486,-122.085585
1,67890,"1 Apple Park Way, Cupertino, CA",37.329729,-122.009922
2,54321,"350 5th Ave, New York, NY",40.748442,-73.985659
